In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
pass_sat=pd.read_csv('Passenger_Satisfaction.csv')
pass_sat.head()

,Unnamed: 0,id,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,...,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,satisfaction
0,0,70172,Male,Loyal Customer,13,Personal Travel,Eco Plus,460,3,4,...,5,4,3,4,4,5,5,25,18.0,neutral or dissatisfied
1,1,5047,Male,disloyal Customer,25,Business travel,Business,235,3,2,...,1,1,5,3,1,4,1,1,6.0,neutral or dissatisfied
2,2,110028,Female,Loyal Customer,26,Business travel,Business,1142,2,2,...,5,4,3,4,4,4,5,0,0.0,satisfied
3,3,24026,Female,Loyal Customer,25,Business travel,Business,562,2,5,...,2,2,5,3,1,4,2,11,9.0,neutral or dissatisfied
4,4,119299,Male,Loyal Customer,61,Business travel,Business,214,3,3,...,3,3,4,4,3,3,3,0,0.0,satisfied


In [3]:
pass_sat=pass_sat.loc[:,~pass_sat.columns.str.contains("^Unnamed")]

In [4]:
pass_sat=pd.get_dummies(pass_sat,columns=pass_sat.select_dtypes(include='object').columns,drop_first=True)

In [5]:
pass_sat.head()

,id,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,...,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,Gender_Male,Customer Type_disloyal Customer,Type of Travel_Personal Travel,Class_Eco,Class_Eco Plus,satisfaction_satisfied
0,70172,13,460,3,4,3,1,5,3,5,...,5,5,25,18.0,True,False,True,False,True,False
1,5047,25,235,3,2,3,3,1,3,1,...,4,1,1,6.0,True,True,False,False,False,False
2,110028,26,1142,2,2,2,2,5,5,5,...,4,5,0,0.0,False,False,False,False,False,True
3,24026,25,562,2,5,5,5,2,2,2,...,4,2,11,9.0,False,False,False,False,False,False
4,119299,61,214,3,3,3,3,4,5,5,...,3,3,0,0.0,True,False,False,False,False,True


In [6]:
pass_sat.isnull().sum()

id                                     0
Age                                    0
Flight Distance                        0
Inflight wifi service                  0
Departure/Arrival time convenient      0
Ease of Online booking                 0
Gate location                          0
Food and drink                         0
Online boarding                        0
Seat comfort                           0
Inflight entertainment                 0
On-board service                       0
Leg room service                       0
Baggage handling                       0
Checkin service                        0
Inflight service                       0
Cleanliness                            0
Departure Delay in Minutes             0
Arrival Delay in Minutes             310
Gender_Male                            0
Customer Type_disloyal Customer        0
Type of Travel_Personal Travel         0
Class_Eco                              0
Class_Eco Plus                         0
satisfaction_sat

In [7]:
pass_sat['Arrival Delay in Minutes'].fillna(pass_sat['Arrival Delay in Minutes'].median(),inplace=True)

In [8]:
pass_sat.to_csv("PASS_SATISFICATION.csv")

### EDA

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier
from sklearn.metrics import classification_report,accuracy_score
from sklearn.preprocessing import StandardScaler,LabelEncoder
from sklearn.model_selection import train_test_split
from mlflow.tracking import MlflowClient
from mlflow.exceptions import MlflowException

In [11]:
X=pass_sat.drop(columns=['satisfaction_satisfied'])
y=pass_sat['satisfaction_satisfied']

In [12]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=50)

In [54]:
mlflow.end_run()

In [36]:
from mlflow.models.signature import infer_signature

In [56]:
import mlflow
import mlflow.sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os
import pandas as pd
from mlflow.models.signature import infer_signature



# Scaling the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Define models and parameters
models = {
    "LogisticRegression": {
        "model": LogisticRegression(max_iter=200),  # Increased max_iter for convergence
        "params": {
            "C": 1.0,
        }
    },
    "RandomForestClassifier": {
        "model": RandomForestClassifier(),
        "params": {
            "n_estimators": 100,
            "max_depth": None,
        }
    },
    "GradientBoostingClassifier": {
        "model": GradientBoostingClassifier(),
        "params": {
            "n_estimators": 100,
            "learning_rate": 0.1,
        }
    }
}

# Set the name of the experiment (This will create the experiment if it doesn't exist)
mlflow.set_experiment("Passenger_Satisfaction_Experiments")
mlflow.set_tracking_uri('http://127.0.0.1:5000/')

# Loop through each model and log each in a separate MLflow run
for model_name, model_data in models.items():
    model = model_data['model']
    params = model_data['params']

    # Set model parameters and fit it
    model.set_params(**params)

    # Start a new MLflow run for each model
    with mlflow.start_run(run_name=model_name):

        # If the model is Logistic Regression, use the scaled data
        if model_name == "LogisticRegression":
            model.fit(X_train_scaled, y_train)
            y_pred = model.predict(X_test_scaled)
        else:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

        # Get classification report metrics
        report = classification_report(y_test, y_pred)
        print(f"Model: {model_name}\n{report}")

        # Convert classification report to dictionary for logging purposes
        report_dic = classification_report(y_test, y_pred, output_dict=True)
        accuracy = report_dic['accuracy']
        precision = report_dic['weighted avg']['precision']
        recall = report_dic['weighted avg']['recall']
        f1score = report_dic['weighted avg']['f1-score']

        # Log model parameters
        mlflow.log_params(params)

        # Log individual metrics
        mlflow.log_metric("Accuracy", accuracy)
        mlflow.log_metric("Precision", precision)
        mlflow.log_metric("Recall", recall)
        mlflow.log_metric("F1-score", f1score)

        # Create signature and input example for model logging
        input_example = pd.DataFrame(X_train[:1], columns=[f'feature_{i}' for i in range(X_train.shape[1])])  # Sample input
        signature = infer_signature(X_train, model.predict(X_train))

        # Log model and signature with example input
        mlflow.sklearn.log_model(model, "model", signature=signature, input_example=input_example)

    print(f"\nLogged {model_name} model in MLflow")

print("\nAll models have been logged.")



2025/03/02 09:36:42 INFO mlflow.tracking.fluent: Experiment with name 'Passenger_Satisfaction_Experiments' does not exist. Creating a new experiment.


Model: LogisticRegression
              precision    recall  f1-score   support

           0       0.90      0.89      0.89      1006
           1       0.89      0.90      0.89       994

    accuracy                           0.89      2000
   macro avg       0.89      0.89      0.89      2000
weighted avg       0.89      0.89      0.89      2000



🏃 View run LogisticRegression at: http://127.0.0.1:5000/#/experiments/772469930172278991/runs/48986045bd044b0e9a187434cb268502
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/772469930172278991

Logged LogisticRegression model in MLflow
Model: RandomForestClassifier
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      1006
           1       0.92      0.97      0.94       994

    accuracy                           0.94      2000
   macro avg       0.94      0.94      0.94      2000
weighted avg       0.94      0.94      0.94      2000



🏃 View run RandomForestClassifier at: http://127.0.0.1:5000/#/experiments/772469930172278991/runs/9b7c821a3f7840d29da52805d67c0f14
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/772469930172278991

Logged RandomForestClassifier model in MLflow
Model: GradientBoostingClassifier
              precision    recall  f1-score   support

           0       0.96      0.92      0.94      1006
           1       0.92      0.96      0.94       994

    accuracy                           0.94      2000
   macro avg       0.94      0.94      0.94      2000
weighted avg       0.94      0.94      0.94      2000



🏃 View run GradientBoostingClassifier at: http://127.0.0.1:5000/#/experiments/772469930172278991/runs/0f2c5be4ca3b494f89be18ecc81c84b5
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/772469930172278991

Logged GradientBoostingClassifier model in MLflow

All models have been logged.


In [58]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
run_ids={
    "LogisticRegression":"48986045bd044b0e9a187434cb268502",
    "RandomForestClassifier":"9b7c821a3f7840d29da52805d67c0f14",
    "GradientBoostingClassifier":"0f2c5be4ca3b494f89be18ecc81c84b5"
}
for model_name,run_id in run_ids.items():
    client.create_model_version(name=model_name,source=f"runs:/{run_ids}/model",run_id=run_id)

print("\n All models registered")

MlflowException: API request to http://127.0.0.1:5000/api/2.0/mlflow/model-versions/create failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow/model-versions/create (Caused by ResponseError('too many 500 error responses'))

In [60]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

# Dictionary containing model names and corresponding run IDs
run_ids = {
    "LogisticRegression": "48986045bd044b0e9a187434cb268502",
    "RandomForestClassifier": "9b7c821a3f7840d29da52805d67c0f14",
    "GradientBoostingClassifier": "0f2c5be4ca3b494f89be18ecc81c84b5"
}

# Loop over each model and register them as model versions
for model_name, run_id in run_ids.items():
    # Specify the source path correctly
    model_uri = f"runs:/{run_id}/model"
    
    # Register the model version in the Model Registry
    client.create_model_version(name=model_name, source=model_uri, run_id=run_id)

print("\nAll models registered.")


MlflowException: API request to http://127.0.0.1:5000/api/2.0/mlflow/model-versions/create failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow/model-versions/create (Caused by ResponseError('too many 500 error responses'))